In [120]:
import pandas as pd
import numpy as np

In [121]:
df = pd.read_csv('all_sets.csv')
df.head()

,set_id,entrant_id,entrant_name,standing,user_id,event_id,source
0,73414311,16280520,LAGr | Nain Jr,1,248637,1115951,startgg
1,73414311,16307025,TACOLOREADO | WORF,2,2235362,1115951,startgg
2,73414312,16310687,BOD | Reina Rata,1,2261338,1115951,startgg
3,73414312,16282031,TwT | Pooters,2,734197,1115951,startgg
4,73414313,16280513,BungeeGumHN,1,2207214,1115951,startgg


In [122]:
elo = pd.read_csv('elo_records.csv')
elo.head()

,user_id,event_id,elo,tier1,tier2,tier3,tier5
0,231154.0,1053190.0,262.819069,0.0,0.0,0.0,1.0
1,722144.0,1053190.0,274.563138,0.0,0.0,0.0,1.0
2,214763.0,1053190.0,242.972738,0.0,0.0,0.0,1.0
3,108449.0,1053190.0,227.719413,0.0,0.0,0.0,1.0
4,2480058.0,1053190.0,183.795282,0.0,0.0,0.0,1.0


In [123]:
default_elo = 200
elo['elo'] = elo.groupby('user_id')['elo'].shift(1).fillna(default_elo)
for tier in ['tier1', 'tier2', 'tier3', 'tier5']:
    elo[tier] = elo.groupby('user_id')[tier].shift(1).fillna(0)
elo.head()

,user_id,event_id,elo,tier1,tier2,tier3,tier5
0,231154.0,1053190.0,200.0,0.0,0.0,0.0,0.0
1,722144.0,1053190.0,200.0,0.0,0.0,0.0,0.0
2,214763.0,1053190.0,200.0,0.0,0.0,0.0,0.0
3,108449.0,1053190.0,200.0,0.0,0.0,0.0,0.0
4,2480058.0,1053190.0,200.0,0.0,0.0,0.0,0.0


In [124]:
df3 = df.merge(elo, on=['user_id', 'event_id'])
df3 = df3.merge(df, on = ['set_id'])
df3 = df3[(df3['standing_y'] != df3['standing_x'])].drop_duplicates(subset=['set_id', 'event_id_x'], keep='first')
df3 = df3.merge(elo, left_on=['user_id_y', 'event_id_y'], right_on=['user_id', 'event_id'])

df3.head()

,set_id,entrant_id_x,entrant_name_x,standing_x,user_id_x,event_id_x,source_x,elo_x,tier1_x,tier2_x,...,user_id_y,event_id_y,source_y,user_id,event_id,elo_y,tier1_y,tier2_y,tier3_y,tier5_y
0,73412191,16309407,Ironknit,2,1373306,1094685,startgg,354.924374,0.0,0.0,...,1253960,1094685,startgg,1253960.0,1094685.0,345.983969,0.0,0.0,0.0,7.0
1,73412192,16033627,SimplyEgo,1,1253960,1094685,startgg,345.983969,0.0,0.0,...,1373306,1094685,startgg,1373306.0,1094685.0,354.924374,0.0,0.0,0.0,25.0
2,73412190,15977585,Mashedpotatoes,2,2291592,1094685,startgg,325.234198,0.0,0.0,...,1373306,1094685,startgg,1373306.0,1094685.0,354.924374,0.0,0.0,0.0,25.0
3,73412222,15977585,Mashedpotatoes,2,2291592,1094685,startgg,325.234198,0.0,0.0,...,1253960,1094685,startgg,1253960.0,1094685.0,345.983969,0.0,0.0,0.0,7.0
4,73412221,16100806,Deznere,2,249900,1094685,startgg,361.670839,0.0,0.0,...,1253960,1094685,startgg,1253960.0,1094685.0,345.983969,0.0,0.0,0.0,7.0


In [125]:
filter_cols = ['set_id','event_id_x', 'elo_x', 'elo_y', 'tier1_x', 'tier1_y', 'tier2_x', 'tier2_y',
               'tier3_x', 'tier3_y', 'tier5_x', 'tier5_y', 'standing_x']

df3 = df3[filter_cols]
df3['result'] = 0
df3.loc[df3['standing_x'] == 1, 'result'] = 0
df3.loc[df3['standing_x'] == 2, 'result'] = 1

df3['all_comps_x'] = df3['tier1_x'] + df3['tier2_x'] + df3['tier3_x'] + df3['tier5_x']
df3['all_comps_y'] = df3['tier1_y'] + df3['tier2_y'] + df3['tier3_y'] + df3['tier5_y']

df3.drop(columns=['standing_x'], inplace=True)
df3.head()

,set_id,event_id_x,elo_x,elo_y,tier1_x,tier1_y,tier2_x,tier2_y,tier3_x,tier3_y,tier5_x,tier5_y,result,all_comps_x,all_comps_y
0,73412191,1094685,354.924374,345.983969,0.0,0.0,0.0,0.0,0.0,0.0,25.0,7.0,1,25.0,7.0
1,73412192,1094685,345.983969,354.924374,0.0,0.0,0.0,0.0,0.0,0.0,7.0,25.0,0,7.0,25.0
2,73412190,1094685,325.234198,354.924374,0.0,0.0,0.0,0.0,0.0,0.0,7.0,25.0,1,7.0,25.0
3,73412222,1094685,325.234198,345.983969,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,1,7.0,7.0
4,73412221,1094685,361.670839,345.983969,0.0,0.0,0.0,0.0,0.0,0.0,22.0,7.0,1,22.0,7.0


In [126]:
# Linear Regression Model
from sklearn.linear_model import LinearRegression
model = LinearRegression()

X = df3.copy()[['all_comps_x', 'all_comps_y', 'elo_x', 'elo_y', 'tier1_x', 'tier1_y']]
y = df3['result']

In [127]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y.astype(int), shuffle = True, train_size=0.3)

model.fit(X_train, y_train)

LinearRegression()

In [128]:
from sklearn.metrics import mean_squared_error, r2_score
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)

print('The r2 is: ', r2)
print('The rmse is: ', rmse)


The r2 is:  0.08127411735351953
The rmse is:  0.4684817107142791


In [129]:
# Linear Regression Model
from sklearn.linear_model import Lasso
model = Lasso(alpha=0.1)

model.fit(X_train, y_train)

from sklearn.metrics import mean_squared_error, r2_score
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)

print('The r2 is: ', r2)
print('The rmse is: ', rmse)

The r2 is:  0.08098824926973103
The rmse is:  0.46855459075131367


In [130]:
from sklearn.linear_model import TweedieRegressor
reg = TweedieRegressor(power=1, alpha=0.5, link='log')
reg.fit(X_train, y_train.astype(int))

print(reg.coef_)
print(reg.intercept_)

y_preds = [1 if x > 0.5 else 0 for x in reg.predict(X_test)]

[-0.00051614  0.00015504 -0.00385218  0.00339212 -0.00221586 -0.00549376]
-0.8296614514503683


In [131]:
# import the metrics class
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test.astype(int), y_preds)
cnf_matrix

array([[24873,  1471],
       [13956,  3215]], dtype=int64)

In [132]:
from sklearn.metrics import classification_report
target_names = ['P1 win', 'P2 win']
print(classification_report(y_test, y_preds, target_names=target_names))

              precision    recall  f1-score   support

      P1 win       0.64      0.94      0.76     26344
      P2 win       0.69      0.19      0.29     17171

    accuracy                           0.65     43515
   macro avg       0.66      0.57      0.53     43515
weighted avg       0.66      0.65      0.58     43515

